# PostgreSQL + Python

In [ ]:
#!pip install SQLAlchemy
#!pip install psycopg2-binary

In [ ]:
import os
import pandas as pd
from sqlalchemy import text, create_engine

**PART 1**: Connect to the database

**PART 2**: Run queries

### PART 1: Connect to the database

#### 1.1. In order to connect to any database, we need 5 things

1. Host
2. Username
3. Password
4. Port
5. Database

`psql -U username -h host -p port -d database`

In [ ]:
HOST = 'localhost'
USERNAME = 'marija'
PORT = '5432'
DB = 'northwind'

Set your postgres password as environment variable.

On mac/linux:

1. Navigate to your home directory (type `cd` in your terminal)
2. Open your `.bashrc` file in the text editor of your choice
3. Add the following line to your `.bashrc` file: `export PG_PASSWORD='*****'`
4. After closing `.bashrc` file type `source ~/.bashrc` in the terminal
5. Open a new jupyter notebook session 

On Windows: 
1. Follow the instructions above if you have / want to create .bashrc file, or
2. Follow this post for how to set your environment variables: https://www.alphr.com/environment-variables-windows-10/

In [ ]:
PASSWORD = os.getenv('PG_PASSWORD')

#### 1.2. Create a connection string ("URL" for our database)

In [ ]:
conn_string = f'postgresql://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DB}'

In [ ]:
conn_string_mac = f'postgresql://{HOST}:{PORT}/{DB}'

#### 1.3. Connect to your `northwind` database

In [ ]:
engine = create_engine(conn_string)

#### 1.4. Execute your first query from Python!

In [ ]:
query = '''
CREATE TABLE IF NOT EXISTS orders_berlin (
    order_id INT PRIMARY KEY,
    customer_id TEXT,
    ship_name TEXT
)
'''

In [ ]:
engine.execute(query)

Now go check in your database to make sure it worked!

### Part 2: Run queries

#### 2.1. SQL + Pandas

We'll use three of the Pandas functions that are similar to things you've already seen, `.to_sql`, `.read_sql_table`, `.read_sql_query`.

Let's first load some data from our northwind `.csv`s.

In [ ]:
orders = pd.read_csv('./northwind_data_clean/data/orders.csv')

In [ ]:
orders.head()

In [ ]:
orders_berlin = orders[orders['shipCity'] == 'Berlin'][['orderID', 'customerID', 'shipName']]

In [ ]:
orders_berlin

`.to_sql`

In [ ]:
orders_berlin.to_sql('orders_berlin', engine, if_exists='replace', index=False)

* Instead of replacing, can also `append` or `fail`.
* Use `method='multi'` when sending a large dataframe on e.g. Amazon Redshift

Look at your table description in `psql`. What do you notice? 

In [ ]:
engine.execute('ALTER TABLE orders_berlin ADD PRIMARY KEY ("orderID")')

`.read_sql_query`

In [ ]:
query = '''
    SELECT order_id, customer_id, ship_name
    FROM orders
    WHERE ship_city = 'Berlin'
'''

In [ ]:
orders_berlin_query = pd.read_sql(query, engine)

In [ ]:
orders_berlin_query.set_index('order_id', inplace=True)

In [ ]:
orders_berlin_query

`read_sql_table`

In [ ]:
orders_berlin_table = pd.read_sql_table('orders_berlin', engine)

In [ ]:
orders_berlin_table.set_index('orderID', inplace=True)

In [ ]:
orders_berlin_table

#### 2.2. Running queries directly in the database

In [ ]:
engine.execute(query)

In [ ]:
list(engine.execute(query))

#### 2.3. Parametrized queries

##### Bad way!

String formatting the query

In [ ]:
city = 'Berlin'

In [ ]:
query_param1 = '''
    SELECT order_id, customer_id, shipped_date
    FROM orders
    WHERE ship_city = '%s'
''' %city

In [ ]:
list(engine.execute(query_param1))

##### Good way!

Passing the parameter to `.execute`.

In [ ]:
query_param2 = text('''
    SELECT order_id, customer_id, shipped_date 
    FROM orders 
    WHERE ship_city = :city
    AND ship_country = :country
''')

In [ ]:
param_dict = {'city': 'Berlin', 'country': 'Germany'}

In [ ]:
list(engine.execute(query_param2, param_dict))

##### Exercise: Modify the parameter below to perform SQL injection / delete one of your tables

In [ ]:
param = 'Misc'

Solution:

In [ ]:
param = "Misc'); DROP TABLE orders_berlin;--"

Explanation here: https://www.explainxkcd.com/wiki/index.php/327:_Exploits_of_a_Mom

In [ ]:
query_injection = '''
    INSERT INTO categories (category_id, category_name)
    VALUES (109, '%s')
''' %param

(Because `category_id` is a primary key, you'll have to keep changing the value inserted for `category_id` (e.g. 101, 102...) as you're debugging.)

In [ ]:
engine.execute(query_injection)

This was just a quick introduction into sqlalchemy. In its full functionality it is a very powerful toolkit. If you are interested in learning more, or are working with databases in Python, start here: https://docs.sqlalchemy.org/en/13/core/tutorial.html

Things you can expect: `Table`, `Column`, `ForeignKey` objects, `.insert()`, `.select()`, `.join()` methods.